## Import dependencies and data sets

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # or any {‘0’, ‘1’, ‘2’}
import tensorflow as tf

In [ ]:
!pip list

In [ ]:
hasGPUSupport = tf.test.is_built_with_cuda()
gpuList = tf.config.list_physical_devices('GPU')
print("Tensorflow Compiled with CUDA/GPU Support:", hasGPUSupport)
print("Tensorflow can access", len(gpuList), "GPU")
print("Accessible GPUs are:")
print(gpuList)

In [ ]:
for gpu in gpuList:
    tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

In [ ]:
#bringing in fashion mnist dataset
ds = tfds.load('fashion_mnist',split='train')

In [ ]:
ds.as_numpy_iterator().next().keys()

## Visualizing Data Set

In [ ]:
import numpy as np

In [ ]:
#Setting up data iterator
data_iterator = ds.as_numpy_iterator()

In [ ]:
#Getting data out of the pipeline
data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4):
    batch = data_iterator.next()
    ax[idx].imshow(np.squeeze(batch['image']))
    ax[idx].title.set_text(batch['label'])

In [5]:
#Scale and return images only
def scale_images(data):
    image = data['image']
    return image/255

## Preping the Dataset for Training

In [3]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

In [6]:
#Reload the dataset
ds = tfds.load('fashion_mnist',split='train')
#Running the dataset through the scale_images preprocessing step
ds = ds.map(scale_images)
#cache the dataset from that batch
ds = ds.cache()
#shuffle it up
ds = ds.shuffle(60000)
#batch into 128 images per sample
ds = ds.batch(128)
#Reduces the likelihood of bottle necking
ds = ds.prefetch(64)

# Building the Neural Network

## Building the Generator

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D

In [18]:
def build_generator():
    model = Sequential()
    
    # Takes in random values and reshape it to 7x7x128
    #model.add(Dense(7*7*128, input_dim=128))
    model.add(Dense(7*7*128, input_shape=(1,)))
    #model.add(Flatten())
    model.add(LeakyReLU(0.2))
    model.add(Reshape((7,7,128)))
    
    # Up-sampling block 1
    model.add(UpSampling2D())
    model.add(Conv2D(128,5,padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Up-sampling block 2
    model.add(UpSampling2D())
    model.add(Conv2D(128,5,padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Convolutional block 1
    model.add(Conv2D(128,4,padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Convolutional block 2
    model.add(Conv2D(128,4,padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Conv Layer to get to one Channel
    model.add(Conv2D(1,4,padding='same', activation='sigmoid'))
    
    return model

In [ ]:
def test_model():
    model = Sequential()
    
    # Takes in random values and reshape it to 7x7x128
    model.add(Dense(32, input_shape=(3,)))
    model.add(Flatten())
    model.add(Dense(1))
    
    return model

In [ ]:
test_model = test_model()
test_model.summary()
input_x = np.random.randn(28,3)
output_y = test_model(input_x)

In [19]:
test_model = build_generator()
test_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 6272)              12544     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 6272)              0         
                                                                 
 reshape_5 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d (UpSampling2  (None, 14, 14, 128)       0         
 D)                                                              
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 128)       409728    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 14, 14, 128)       0         
                                                      

In [20]:
input_x = np.random.randn(128,1)

#print(f"input_x = {input_x}")
print(f"input_x shape = {input_x.shape}")

input_x shape = (128, 1)


In [22]:
img = test_model.predict(input_x)
img

: 